# Keras Recurrent Neural Network

In [40]:
import numpy as np
import pandas as pd

import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, RNN
from keras.optimizers import RMSprop
import keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [41]:
# x_train = 

In [42]:
# Tensorflow test

# import tensorflow as tf
# import numpy as np
# # Create 100 phony x, y data points in NumPy, y = x * 0.1 + 0.3
# x_data = np.random.rand(100).astype(np.float32)
# y_data = x_data * 0.1 + 0.3
# # Try to find values for W and b that compute y_data = W * x_data + b
# # (We know that W should be 0.1 and b 0.3, but Tensorflow will
# # figure that out for us.)
# W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
# b = tf.Variable(tf.zeros([1]))
# y = W * x_data + b
# # Minimize the mean squared errors.
# loss = tf.reduce_mean(tf.square(y - y_data))
# optimizer = tf.train.GradientDescentOptimizer(0.5)
# train = optimizer.minimize(loss)
# # Before starting, initialize the variables.  We will 'run' this first.
# init = tf.initialize_all_variables()
# # Launch the graph.
# sess = tf.Session()
# sess.run(init)
# # Fit the line.
# for step in range(201):
#     sess.run(train)
#     if step % 20 == 0:
#         print(step, sess.run(W), sess.run(b))
# # Learns best fit is W: [0.1], b: [0.3]

In [43]:
# Keras test

# '''Trains a simple deep NN on the MNIST dataset.
# Gets to 98.40% test accuracy after 20 epochs
# (there is *a lot* of margin for parameter tuning).
# 2 seconds per epoch on a K520 GPU.
# '''

# from __future__ import print_function

# import keras
# from keras.datasets import mnist
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.optimizers import RMSprop

# batch_size = 128
# num_classes = 10
# epochs = 20

# # the data, split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# print(x_train[0])
# x_train = x_train.astype('float32')
# x_train /= 255
# print(x_train[0])
# x_train = x_train.reshape(60000, 784)
# x_test = x_test.reshape(10000, 784)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# print(x_train.shape[0], 'train samples')
# print(x_test.shape[0], 'test samples')

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Dense(512, activation='relu', input_shape=(784,)))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(num_classes, activation='softmax'))

# model.summary()

# model.compile(loss='categorical_crossentropy',
#               optimizer=RMSprop(),
#               metrics=['accuracy'])

# history = model.fit(x_train, y_train,
#                     batch_size=batch_size,
#                     epochs=epochs,
#                     verbose=1,
#                     validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])


In [44]:
# # First, let's define a RNN Cell, as a layer subclass.

# class MinimalRNNCell(keras.layers.Layer):

#     def __init__(self, units, **kwargs):
#         self.units = units
#         self.state_size = units
#         super(MinimalRNNCell, self).__init__(**kwargs)

#     def build(self, input_shape):
#         self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
#                                       initializer='uniform',
#                                       name='kernel')
#         self.recurrent_kernel = self.add_weight(
#             shape=(self.units, self.units),
#             initializer='uniform',
#             name='recurrent_kernel')
#         self.built = True

#     def call(self, inputs, states):
#         prev_output = states[0]
#         h = K.dot(inputs, self.kernel)
#         output = h + K.dot(prev_output, self.recurrent_kernel)
#         return output, [output]

# # Let's use this cell in a RNN layer:

# cell = MinimalRNNCell(32)
# x = keras.Input((None, 5))
# layer = RNN(cell)
# y = layer(x)

# # Here's how to use the cell to build a stacked RNN:

# cells = [MinimalRNNCell(32), MinimalRNNCell(64)]
# x = keras.Input((None, 5))
# layer = RNN(cells)
# y = layer(x)

In [45]:
x_train_df = pd.read_csv("preprocessed_train_split_feat.csv")
print("Training data shape:", x_train_df.shape)

Training data shape: (207, 16)


In [46]:
x_test_df = pd.read_csv("preprocessed_validation_split_feat.csv")

print("Test data shape:", x_test_df.shape)

Test data shape: (60, 16)


In [47]:
y_train_df = pd.read_csv("train_split_label.csv").Label
print("Training label shape:", y_train_df.shape)

Training label shape: (207,)


In [48]:
y_test_df = pd.read_csv("validation_split_label.csv").Label
print("Test label shape:", y_test_df.shape)

Test label shape: (60,)


In [49]:
# def create_rnn_model():
#     # Input layers
#     teacher_number_of_previously_posted_projects = keras.layers.Input(shape=(1,), name="teacher_number_of_previously_posted_projects")
#     project_title = keras.layers.Input(shape=(MAX_PROJECT_TITLE_SEQ_LEN,), name="project_title")
#     project_essay = keras.layers.Input(shape=(MAX_PROJECT_ESSAY_SEQ_LEN,), name="project_essay")
#     #project_resource_summary = keras.layers.Input(shape=(MAX_PROJECT_RESOURCE_SUMMARY_SEQ_LEN,), name="project_resource_summary")
    
#     # Embedding layers
#     #emb_project_title = keras.layers.Embedding(MAX_PROJECT_TITLE, 25)(project_title)
#     #emb_project_essay = keras.layers.Embedding(MAX_PROJECT_ESSAY, 50)(project_essay)
#     emb_layer = keras.layers.Embedding(MAX_TEXT, 50)
#     emb_project_title = emb_layer(project_title)
#     emb_project_essay = emb_layer(project_essay)
    
#     # RNN layers
#     rnn_project_title = keras.layers.GRU(8, activation="relu")(emb_project_title)
#     rnn_project_essay = keras.layers.GRU(16, activation="relu")(emb_project_essay)
#     #rnn_project_resource_summary = keras.layers.GRU(16, activation="relu")(emb_project_resource_summary)
    
#     # Merge all layers into one
#     x = keras.layers.concatenate([teacher_number_of_previously_posted_projects,
#                                  rnn_project_title,
#                                  rnn_project_essay,
#                                  #rnn_project_resource_summary,
#                                  ])
    
#     # Dense layers
#     #x = keras.layers.Dense(128, activation="relu")(x)

#     # Output layers
#     output = keras.layers.Dense(1, activation="sigmoid")(x)
    
#      return keras.models.Model(
#         inputs=[teacher_number_of_previously_posted_projects,
#                 project_title,
#                 project_essay,
#                 #project_resource_summary,
#                ],
#         outputs=output)

# rnn_model = create_rnn_model()

In [230]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='sgd', alpha=1e-5,
                    hidden_layer_sizes=(12, 6), random_state=2)

clf.fit(x_train_df, y_train_df)                         

/home/kaj/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(12, 6), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=2, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [231]:
pred = clf.predict(x_test_df)
pred

array(['standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing',
       'standing', 'standing', 'standing', 'standing', 'standing'],
      dtype='<U12')

In [232]:
def print_accuracy(pred, control):
    total = 0
    correct = 0
    for i in range(len(pred)):
        total += 1
        if pred[i] == control[i]:
            correct += 1

    print('Accuracy:',correct*100/total,'%')

In [233]:
from sklearn import metrics
print(metrics.accuracy_score(pred, y_test_df))

0.5666666666666667
